In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
sys.path.append(os.path.join(sys.path[0],'..','tools'))
import torch  # put this before scipy import
from mattnet import MattNet

%pylab inline
import matplotlib.pyplot as plt

from PIL import Image
import argparse
import numpy as np

from scipy.misc import imread, imresize
from itertools import product

print(torch.cuda.current_device())
print(torch.cuda.device_count())

Populating the interactive namespace from numpy and matplotlib
0
1


In [2]:
# arguments
parser = argparse.ArgumentParser()
parser.add_argument('--dataset', type=str, default='refcoco', help='dataset name: refclef, refcoco, refcoco+, refcocog')
parser.add_argument('--splitBy', type=str, default='unc', help='splitBy: unc, google, berkeley')
parser.add_argument('--model_id', type=str, default='mrcn_cmr_with_st', help='model id name')
args = parser.parse_args('')

In [3]:
# MattNet
mattnet = MattNet(args)

/imatge/alba.herrera/maskMattnet/MAttNet/venv2/local/lib/python2.7/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'layers.joint_match.JointMatching' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/imatge/alba.herrera/maskMattnet/MAttNet/venv2/local/lib/python2.7/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/imatge/alba.herrera/maskMattnet/MAttNet/venv2/local/lib/python2.7/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.modu

MatNet [refcoco_unc's mrcn_cmr_with_st] loaded in 3.96 seconds.
Using config:
{'ANCHOR_RATIOS': [0.5, 1, 2],
 'ANCHOR_SCALES': [4, 8, 16, 32],
 'DATA_DIR': '/imatge/alba.herrera/maskMattnet/MAttNet/pyutils/mask-faster-rcnn/data',
 'EXP_DIR': 'res101',
 'MASK_SIZE': 14,
 'MATLAB': 'matlab',
 'MOBILENET': {'DEPTH_MULTIPLIER': 1.0,
               'FIXED_LAYERS': 5,
               'REGU_DEPTH': False,
               'WEIGHT_DECAY': 4e-05},
 'PIXEL_MEANS': array([[[102.9801, 115.9465, 122.7717]]]),
 'POOLING_ALIGN': False,
 'POOLING_MODE': 'crop',
 'POOLING_SIZE': 7,
 'RESNET': {'FIXED_BLOCKS': 1, 'MAX_POOL': False},
 'RNG_SEED': 3,
 'ROOT_DIR': '/imatge/alba.herrera/maskMattnet/MAttNet/pyutils/mask-faster-rcnn',
 'TEST': {'BBOX_REG': True,
          'HAS_RPN': True,
          'MAX_SIZE': 1000,
          'MODE': 'nms',
          'NMS': 0.3,
          'PROPOSAL_METHOD': 'gt',
          'RPN_NMS_THRESH': 0.7,
          'RPN_POST_NMS_TOP_N': 300,
          'RPN_PRE_NMS_TOP_N': 6000,
          

../tools/../pyutils/mask-faster-rcnn/lib/model/config.py:362: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  yaml_cfg = edict(yaml.load(f))


Done (t=0.68s)
creating index...
index created!
pretrained-model loaded from [../tools/../lib/mrcn/../../pyutils/mask-faster-rcnn/output/res101/coco_2014_train_minus_refer_valtest+coco_2014_valminusminival/notime/res101_mask_rcnn_iter_1250000.pth].
Mask R-CNN: imdb[coco_minus_refer], tag[notime], id[res101_mask_rcnn_iter_1250000] loaded in 5.64 seconds.


In [7]:
def save_masks(pred_path, masks):
    h, w = masks[0].shape
    prediction = np.zeros((h, w))
 
    for i in reversed(range(len(masks))):
        # set the current k value
        current_k = i+1
        # change ones for actual k value
        prediction[np.array(masks[i]) == 1] = current_k
        
    a = Image.fromarray(prediction.astype(np.uint8), mode="P")
    a.putpalette(PALETTE)
    a.save(os.path.join(pred_path, str(0).zfill(5) + '.png'))

In [8]:
# DAVIS query information
davis = ['16'] # , '17']
annot = ['annot1', 'annot2']

# image path
IMAGE_DIR = '../../datasets/'
PRED_DIR = '../data/'

# expression annotations
ANNOT_DIR = '../../datasets/davis_text_annotations/'
expr = ''

# save masks palette
PALETTE = [0, 0, 0, 128, 0, 0, 0, 128, 0, 128, 128, 0, 0, 0, 128, 128, 0, 128, 0, 128, 128, 128, 128, 128, 64,
           0, 0, 191, 0, 0, 64, 128, 0, 191, 128, 0, 64, 0, 128]


In [9]:
for d, a in product(davis, annot):
    
    annotation_file = 'Davis' + d + '_' + a + '.txt'
    # Open the file with read only permit
    f = open( ANNOT_DIR + annotation_file)
    for line in f:
        parts = line.split(' ')
        query = parts[0]
        img_path = os.path.join(IMAGE_DIR, 'DAVIS' + d,'JPEGImages/480p/', query, str(0).zfill(5) + '.jpg')
        # forward image
        img_data = mattnet.forward_image(img_path, nms_thresh=0.3, conf_thresh=0.50)
        # comprehend expression
        expr = ' '.join(parts[2:]).replace('"', '').strip()
        entry = mattnet.comprehend(img_data, expr)
        pred_path = os.path.join(PRED_DIR, 'DAVIS' + d + '_480p_masks', a, query)
        save_masks(pred_path, masks=[entry['pred_mask']])
    f.close()
